In [ ]:
import numpy as np
import cv2
import os
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
!curl -o haarcascade_frontalface_alt.xml  https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_alt.xml 

In [ ]:
#Init Camera
cap = cv2.VideoCapture(0)

# Face Detection
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

In [ ]:
skip = 0
face_data = []
dataset_path = './data/'
os.makedirs(dataset_path, exist_ok=True)

In [ ]:
file_name = input("Enter the name of the person : ")
while True:
    ret,frame = cap.read()

    if ret==False:
        continue

    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)


    faces = face_cascade.detectMultiScale(frame,1.3,5)
    if len(faces)==0:
        continue

    faces = sorted(faces,key=lambda f:f[2]*f[3])

    # Pick the last face (because it is the largest face acc to area(f[2]*f[3]))
    for face in faces[-1:]:
        x,y,w,h = face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

        #Extract (Crop out the required face) : Region of Interest
        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))

        skip += 1
        if skip%10==0:
            face_data.append(face_section)
            print(len(face_data))


    cv2.imshow("Frame",frame)
    cv2.imshow("Face Section",face_section)

    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break

# Convert our face list array into a numpy array
face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0],-1))
print(face_data.shape)

# Save this data into file system
np.save(os.path.join(dataset_path, file_name+'.npy'),face_data)
print("Data Successfully save at "+dataset_path+file_name+'.npy')

cap.release()
cv2.destroyAllWindows()

In [ ]:
#Init Camera
cap = cv2.VideoCapture(0)

# Face Detection
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

In [ ]:
skip = 0
dataset_path = './data/'

In [ ]:
face_data = [] 
labels = []

In [ ]:
class_id = 0
names = {}

In [ ]:
# Data Preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        #Create a mapping btw class_id and name
        names[class_id] = fx[:-4]
        print("Loaded "+fx)
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)

        #Create Labels for the class
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

In [ ]:
face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(labels,axis=0)#.reshape((-1,1))

In [ ]:
print(face_dataset.shape)
print(face_labels.shape)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=5)

In [ ]:
clf.fit(face_dataset, face_labels)

In [ ]:
while True:
    ret,frame = cap.read()
    if ret == False:
        continue

    faces = face_cascade.detectMultiScale(frame,1.3,5)
    if(len(faces)==0):
        continue

    for face in faces:
        x,y,w,h = face

        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))

        out = clf.predict(face_section.reshape(1,-1))
        dist = clf.kneighbors(face_section.reshape(1,-1))[0][0][0]

        pred_name = f'{names[int(out)]} {dist:.0f}'
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("Faces",frame)

    key = cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

https://www.kaggle.com/devendra45/movies-similarity

In [ ]:
df_movies = pd.read_csv('movies.csv')
df_movies.head()

In [ ]:
df_movies.shape

In [ ]:
vect = TfidfVectorizer()

In [ ]:
nn = NearestNeighbors(n_neighbors=5)

In [ ]:
nn.fit(vect.fit_transform(df_movies['wiki_plot']))

In [ ]:
def get_top_films(text):
    vec = vect.transform([text])
    
    dist, neigh = nn.kneighbors(vec)
    print(neigh)
    res = df_movies.loc[neigh[0]]
    return res[['title', 'genre']]

In [ ]:
get_top_films('crime and drugs')

In [ ]:
get_top_films('violence protest march')

In [ ]:
get_top_films('love affair hate')

In [ ]:
get_top_films('fight action')